In [40]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', as_frame=False, parser='auto')
X, y = mnist.data, mnist.target
X_train, X_val, X_test, y_train, y_val, y_test =  X[:50000], X[50000:60000], X[60000:], y[:50000], y[50000:60000], y[60000:]

In [48]:
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier

svc_clf = LinearSVC(max_iter=100)
rnd_fst_clf = RandomForestClassifier(n_estimators=100)
extra_tree_clf = ExtraTreesClassifier(n_estimators=100)
mlp_clf = MLPClassifier()

estimators = [svc_clf, rnd_fst_clf, extra_tree_clf, mlp_clf]
for estimator in estimators:
    print('Training estimator: ', estimator)
    estimator.fit(X_train, y_train)
    
[estimator.score(X_val, y_val) for estimator in estimators]

Training estimator:  LinearSVC(max_iter=100)


f:\Projects\Coding-RasPi\HOML\homl-venv\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Training estimator:  RandomForestClassifier()
Training estimator:  ExtraTreesClassifier()
Training estimator:  MLPClassifier()


[0.8831, 0.9723, 0.9736, 0.965]

In [49]:
from sklearn.ensemble import StackingClassifier, VotingClassifier

est = [
    ('svc', svc_clf), 
    ('rnd_fst', rnd_fst_clf),
    ('extra', extra_tree_clf), 
    ('mlp', mlp_clf)]

voting_clf = VotingClassifier(est)
voting_clf.fit(X_train, y_train)
voting_clf.score(X_val, y_val)

f:\Projects\Coding-RasPi\HOML\homl-venv\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.9758

In [50]:
y_val_encoded = y_val.astype(np.int64)
[estimator.score(X_val, y_val_encoded) for estimator in voting_clf.estimators_]

[0.8795, 0.9723, 0.9741, 0.9683]

In [57]:
svc_trained = voting_clf.named_estimators_.pop("svc")
voting_clf.estimators_.remove(svc_trained)
voting_clf.score(X_val, y_val)


KeyError: 'svc'

In [60]:
voting_clf.voting = "soft"
voting_clf.score(X_val, y_val)

0.9736

In [63]:
voting_clf.voting = 'hard'
voting_clf.score(X_test, y_test), [estimator.score(X_test, y_test.astype(np.int64)) for estimator in voting_clf.estimators_]

(0.973, [0.9677, 0.9705, 0.9659])